In [1]:
import numpy as np
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

import warnings
warnings.filterwarnings("ignore")

import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Load the pretrained model

In [2]:
multi = ['seq','open','loop'] #Multimodal data used as input in the model
pretrain = './pth/pretrain_%s.pth' % ('+'.join(multi)) #Path of pretrained model
channel1 = 128 * (len(multi)+1) if 'seq' in multi else 128 * len(multi)

In [3]:
import create
clf = create.model.layer.create(multi=multi, channel1=channel1).cuda()
checkpoint = torch.load(pretrain)
clf.load_state_dict(checkpoint['model'])
clf.eval()

create(
  (vqvae): VQVAE(
    (enc1): Encoder1(
      (conv1): Sequential(
        (0): Conv1d(4, 256, kernel_size=(8,), stride=(1,))
        (1): ReLU(inplace=True)
        (2): MaxPool1d(kernel_size=5, stride=4, padding=0, dilation=1, ceil_mode=False)
        (3): Dropout(p=0.1, inplace=False)
      )
      (layernorm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (conv11): Sequential(
        (0): Conv1d(1, 128, kernel_size=(8,), stride=(1,))
        (1): ReLU(inplace=True)
        (2): MaxPool1d(kernel_size=5, stride=4, padding=0, dilation=1, ceil_mode=False)
        (3): Dropout(p=0.1, inplace=False)
      )
      (layernorm11): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (conv12): Sequential(
        (0): Conv1d(1, 128, kernel_size=(8,), stride=(1,))
        (1): ReLU(inplace=True)
        (2): MaxPool1d(kernel_size=5, stride=4, padding=0, dilation=1, ceil_mode=False)
        (3): Dropout(p=0.1, inplace=False)
      )
      (layernorm12): LayerNorm

# Predicted for novel regions

In [4]:
data_path = './example/'
test_data = []
if 'seq' in multi:
    test_data.append(np.load(data_path+'test_seqs.npy'))
if 'open' in multi:
    test_data.append(np.load(data_path+'test_opens.npy'))
if 'loop' in multi:
    test_data.append(np.load(data_path+'test_loops.npy'))
test_data = np.concatenate(test_data, axis=1)
test_set = TensorDataset(torch.Tensor(test_data))
test_loader = DataLoader(test_set, batch_size=1024, shuffle=False)
print('The shape of input data: {}'.format(test_data.shape[1:]))

The shape of input data: (6, 1000)


In [5]:
test_score = []
for _, b_x0 in enumerate(test_loader):
    output, _, _, _, _, _ = clf(b_x0[0].cuda())
    test_score.extend(F.softmax(output, 1).cpu().detach().numpy())
test_score = np.array(test_score)
test_pre = [list(x).index(max(x)) for x in test_score]

In [6]:
from collections import Counter
Counter(test_pre)

Counter({0: 211, 1: 207, 2: 200, 3: 198, 4: 184})